### Langkah 1

In [21]:
# Mengimport library
import numpy as np
import pandas as pd
import tensorflow as tf

### Langkah 2

In [22]:
# Load data
dataset = pd.read_csv('../docs/Churn_Modelling.csv')

# Memisah data X dan y
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [23]:
# Menampilkan data X 
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Langkah 3

In [24]:
from sklearn.preprocessing import LabelEncoder # import LabelEncoder dari sklearn.preprocessing
# membuat objek LabelEncoder
le = LabelEncoder()

# Melakukan encoding label male dan female menjadi 0 dan 1
X[:, 2] = le.fit_transform(X[:, 2])

In [25]:
# mengecek data hasil encoding
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### Langkah 4

In [26]:
from sklearn.compose import ColumnTransformer # import ColumnTransformer dari sklearn.compose # import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder # import OneHotEncoder dari sklearn.preprocessing


# Membuat objek ColumnTransformer dengan spesifikasi transformasi
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

# Menggunakan ColumnTransformer untuk mengubah fitur dalam dataset
X = np.array(ct.fit_transform(X))

In [27]:
# Mengecek hasil encoding
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Langkah 5

In [28]:
from sklearn.model_selection import train_test_split # import train_test_split dari sklearn.model_selection

# Membagi dataset menjadi data pelatihan (training) dan data pengujian (testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Langkah 6

In [29]:
# Mengimpor StandardScaler dari sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

# Membuat objek StandardScaler
sc = StandardScaler()

# Melakukan penskalaan fitur pada data pelatihan (X_train)
X_train = sc.fit_transform(X_train)

# Melakukan penskalaan fitur yang sama pada data pengujian (X_test)
X_test = sc.transform(X_test)


## Membuat Model ANN

### Langkah 1

In [30]:
# Membuat objek model Sequential
ann = tf.keras.models.Sequential()

### Langkah 2

In [31]:
# Menambahkan lapisan pertama dengan 6 unit (neuron) dan fungsi aktivasi ReLU
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 3

In [32]:
# Menambahkan lapisan kedua dengan 6 unit (neuron) dan fungsi aktivasi ReLU
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 4

In [33]:
# Menambahkan lapisan ketiga dengan 1 unit (neuron) dan fungsi aktivasi sigmoid
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training Model

### Langkah 1

In [34]:
# Mengkompilasi model dengan optimizer, loss function, dan metrik
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Langkah 2

In [35]:
# Melatih model dengan data pelatihan
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100


250/250 [==============================] - 2s 2ms/step - loss: 0.5908 - accuracy: 0.7514
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4914 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4650 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4397 - accuracy: 0.8043
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8102
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4016 - accuracy: 0.8130
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3892 - accuracy: 0.8174
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3804 - accuracy: 0.8186
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3745 - accuracy: 0.8195
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3698 - accuracy: 0.8206
E

## Membuat prediksi

In [36]:
# Melakukan prediksi pada data tes baru
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 88ms/step
[[False]]


### Prediksi Dengan Data Testing

In [37]:
# Melakukan prediksi pada data tes dan mengubah hasil menjadi True atau False berdasarkan threshold 0.5
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

# Mencetak hasil prediksi dan label sebenarnya dari data tes
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Cek Akurasi dan Confusion Matrix

In [38]:
# Menggunakan metrik untuk evaluasi model
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)

# Mencetak confusion matrix dan akurasi
print(cm)
accuracy_score(y_test, y_pred)

[[1505   90]
 [ 196  209]]


0.857